In [1]:
import re

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
PATH_DATA = "../../../data/"

In [3]:
ls  -lh ../../../data

total 62793344
drwxr-xr-x  2 nikolay  staff    64B  3 май 12:46 arxiv/
-rw-r--r--  1 nikolay  staff   2,8G  3 май 12:47 radius.log.1
-rw-r--r--@ 1 nikolay  staff   356M  3 май 12:38 radius.log.1.gz
-rw-r--r--  1 nikolay  staff   2,9G  3 май 12:47 radius.log.2
-rw-r--r--@ 1 nikolay  staff   365M  3 май 12:39 radius.log.2.gz
-rw-r--r--  1 nikolay  staff   3,0G  3 май 12:47 radius.log.3
-rw-r--r--@ 1 nikolay  staff   372M  3 май 12:40 radius.log.3.gz
-rw-r--r--  1 nikolay  staff   2,8G  3 май 12:47 radius.log.4
-rw-r--r--@ 1 nikolay  staff   355M  3 май 12:41 radius.log.4.gz
-rw-r--r--  1 nikolay  staff   2,9G  3 май 12:48 radius.log.5
-rw-r--r--@ 1 nikolay  staff   363M  3 май 12:41 radius.log.5.gz
-rw-r--r--  1 nikolay  staff   3,0G  3 май 12:48 radius.log.6
-rw-r--r--@ 1 nikolay  staff   372M  3 май 12:42 radius.log.6.gz
-rw-r--r--  1 nikolay  staff   2,9G  3 май 12:48 radius.log.7
-rw-r--r--@ 1 nikolay  staff   364M  3 май 12:43 radius.log.7.gz
-rw-r--r--  1 nikolay  staff   3,0G  3 м

In [4]:
with open(PATH_DATA + 'radius.log.1', 'rb') as f:
    rows = f.readlines()

rows = [str(row)[2:][:-3] for row in tqdm(rows)]

In [5]:
len_date = len("Apr 30 00:11:25")
print(len_date)

15


In [83]:
def unlist(x: list):
    if len(x) > 0:
        return x[0]
    else:
        return ''

In [119]:
re.findall(r"'(.+)' via", row)

['9usLaHXW-9g@dhcp']

In [123]:
columns = [
    'date', 'rad/nfs', 'radiusd', 'type_', 'id_сессии', 'flag_мастер_сессия',
    'сессия_аккаунтинга_направления', 'логин_абонента']
res_parse = []
for row in tqdm(rows[:100000]):
    date = row[:len_date]
    rad_nfs = row[(len_date+1): (len_date+4)]
    radiusd = unlist(re.findall(r'radiusd: \[(\w+)\]', row))
    type_ = unlist(re.findall(r'radiusd: \[\w+\] ([.]+)', row))
    ulsk = re.findall(r'ULSK-(\w+)', row)
    flg_master_session = "RADIUS,," in row
    session_accountinga_napravlenia = unlist(re.findall(r'RADIUS,(\d),', row))
    login_abonenta = unlist(re.findall(r"'(.+)' via", row))
    
    res_parse.append([
        date, rad_nfs, radiusd, type_, ulsk, flg_master_session, session_accountinga_napravlenia,
        login_abonenta,
    ])

In [116]:
row

"Apr 30 00:24:15 nfc radiusd: [16249] Billing-Accounting: 165/0 '9usLaHXW-9g@dhcp' via radius@startip_nn - OK -- input (2020.04.30 00:24:14,89.239.189.2:253892170,Virtual-Access,Framed,PPP,Alive,ULSK-BR15233161000000c845b7194404,RADIUS,5,dc:e3:05:7c:47:b0,100.74.214.186,0,382500,6767997,10802772, 89.239.189.2, \\\\0x01\\\\0\\\\0\\\\0x060A49F781, ULSK-B59033SSG0000048bbdb1194404, ULSK-BR15233161000000c845b7194404, dc:e3:05:7c:47:b0) delay 0.317029 "

In [45]:
row

"Apr 30 00:12:32 rad radiusd: [17571] Billing-Accounting: 166/0 'Ov8XWXvH@pppoe' via radius@startip_nn - OK -- input (2020.04.30 00:12:32,89.239.189.1:35775480,Virtual-Access,Framed,PPP,Alive,ULSK-BR022301016000005e854a207150,RADIUS,1,1070378-HDSLAM-TIMOSHKINO atm 0/4/0/23:0.33,89.239.148.242,0,390000,65228652,1169689424, 89.239.189.1, 1070378-HDSLAM-TIMOSHKINO atm 0/4/0/23:0.33, ULSK-B04230SSG0000002cef80207150, ULSK-BR022301016000005e854a207150, 60:a4:4c:dc:2b:5c) delay 0.293691"

In [125]:
df['логин_абонента'].value_counts()

                      26736
9usLLWuM-9W@dhcp        121
9usLLXA9-9s@dhcp        112
Gs9AWvsA@pppoe           96
8H89Ws8HXA8s@pppoe       72
                      ...  
9usLtM9i-WA@dhcp          1
9usLXggg-Wv@dhcp          1
9usLtigi-WW@dhcp          1
AA999H-H@pppoe            1
GbR3yAZh@pppoe            1
Name: логин_абонента, Length: 13076, dtype: int64

In [124]:
df = pd.DataFrame(res_parse, columns=columns)
df.head()

,date,rad/nfs,radiusd,type_,id_сессии,flag_мастер_сессия,сессия_аккаунтинга_направления,логин_абонента
0,Apr 30 00:11:25,rad,17626,,"[BR152331598000004a2463043502, B41033SSG000003...",False,4,9usLLsvL-Wg@dhcp
1,Apr 30 00:11:26,nfc,16233,,"[BR09232112900000f15744056293, B07232SSG000003...",False,4,AAHvH8-W@pppoe
2,Apr 30 00:11:25,rad,03402,,[B04230SSG000003e90bb7217311],False,,
3,Apr 30 00:11:26,nfc,16343,,"[BR09232039100000d66624053175, B35032SSG000004...",False,5,9usLtigW-9t@dhcp
4,Apr 30 00:11:25,rad,17553,,"[BR09232039100000d66624053175, B35032SSG000002...",False,3,9usLtigW-9t@dhcp


In [102]:
rows[0]["le"]

"Apr 30 00:11:25 rad radiusd: [17626] Billing-Accounting: 117/0 '9usLLsvL-Wg@dhcp' via radius@startip_nn - OK -- input (2020.04.30 00:11:25,89.239.189.1:253892158,Virtual-Access,Framed,PPP,Alive,ULSK-BR152331598000004a2463043502,RADIUS,4,ec:4c:4d:ae:a7:30,92.252.187.91,0,315000,13958,750386, 89.239.189.1, \\\\0x01\\\\0\\\\0\\\\0x160A499459, ULSK-B41033SSG0000035ccd3f043502, ULSK-BR152331598000004a2463043502, ec:4c:4d:ae:a7:30) delay 0.211458"

In [104]:
dates.value_counts()

rad    6641323
nfc    1929387
dtype: int64

In [54]:
len_ = len("Apr 30 00:11:25") + 2

In [27]:
for i in rows[-30:]:
    print()
    print(str(i))


May  1 00:11:39 rad radiusd: [03402] huawei_save_sessid: added 84c7718 ULSK-B04230SSG000003682f28110260 0000603009110260 4 1588277499

May  1 00:11:39 rad radiusd: [17665] Billing-Accounting: 85/0 '9usLa8sv-9v@dhcp' via radius@startip_nn - OK -- input (2020.05.01 00:11:39,89.239.189.1:253890951,Virtual-Access,Framed,PPP,Alive,ULSK-BR15233039100000b9512d226928,RADIUS,,88:a6:c6:9d:14:82,92.252.181.140,0,4755001,6950081365,83887977081, 89.239.189.1, \\0\\0x04\\0x05$\\0\\0x05\\nI\\0xf3E, ULSK-BR15233039100000b9512d226928, 88:a6:c6:9d:14:82) delay 0.343735

May  1 00:11:39 nfc radiusd: [17401] Billing-Accounting: 71/0 '9usLtMAa-9a@dhcp' via radius@startip_nn - OK -- input (2020.05.01 00:11:39,89.239.189.1:253892106,Virtual-Access,Framed,PPP,Alive,ULSK-BR15233154600000f463b8149128,RADIUS,5,f8:e9:03:83:8d:c0,92.252.143.165,0,60000,1195226,1935586, 89.239.189.1, \\0x01\\0\\0\\0x0f0A49EB2F, ULSK-B41033SSG00000401cc84149128, ULSK-BR15233154600000f463b8149128, f8:e9:03:83:8d:c0) delay 0.227381 


In [ ]:
f.re

In [21]:
a

"Apr 30 00:11:25 rad radiusd: [17626] Billing-Accounting: 117/0 '9usLLsvL-Wg@dhcp' via radius@startip_nn - OK -- input (2020.04.30 00:11:25,89.239.189.1:253892158,Virtual-Access,Framed,PPP,Alive,ULSK-BR152331598000004a2463043502,RADIUS,4,ec:4c:4d:ae:a7:30,92.252.187.91,0,315000,13958,750386, 89.239.189.1, \\0x01\\0\\0\\0x160A499459, ULSK-B41033SSG0000035ccd3f043502, ULSK-BR152331598000004a2463043502, ec:4c:4d:ae:a7:30) delay 0.211458\n"

In [17]:
a = pd.read_csv(PATH_DATA + 'radius.log.1')

ParserError: Error tokenizing data. C error: Expected 20 fields in line 3177, saw 21
